In [19]:
#imports
import os
from google.cloud import firestore
import pandas as pd

In [20]:
# Set the environment variable to use the local Firestore emulator
os.environ["FIRESTORE_EMULATOR_HOST"] = "localhost:4000"

In [11]:
# Create a new client
db = firestore.Client(project="peakflo")

In [21]:
# Perform the query
invoices_ref = db.collection('invoices')
query = invoices_ref.where('companyPayerName', '==', 'Aaron Co 2')

In [14]:
# Execute the query and get the results
results = query.stream()

In [22]:
# Execute the query and get the results
results = query.stream()

In [23]:
# Collect the data into a list of dictionaries
data = []
for doc in results:
    doc_data = doc.to_dict()
    doc_data['id'] = doc.id  # Add the document ID to the data
    data.append(doc_data)

PermissionDenied: 403 Missing or insufficient permissions.